## Visualise a pose from AMASS as SMPL-H model

The HumanML3D conversion swaps the z and y axis, making the human face towards negative z. This is not a valid rotation and mirrors the motion on the x-plane. I think this is why they proceed to multiply the x component with -1 in their script and swap left and right in their script.

Note, that AMASS data comes with a offset to place the human feet at the ground plane.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm import tqdm

from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [2]:
from human_body_prior.body_model.body_model import BodyModel

male_bm_path = "./body_models/smplh/male/model.npz"
male_dmpl_path = "./body_models/dmpls/male/model.npz"

female_bm_path = "./body_models/smplh/female/model.npz"
female_dmpl_path = "./body_models/dmpls/female/model.npz"

num_betas = 10  # number of body parameters
num_dmpls = 8  # number of DMPL parameters

male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=male_dmpl_path).cuda()
faces = c2c(male_bm.f)

female_bm = BodyModel(
    bm_fname=female_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=female_dmpl_path
).cuda()

In [4]:
amass_sample = "amass_data/BMLmovi/BMLmovi/Subject_24_F_MoSh/Subject_24_F_11_poses.npz"

ex_fps = 20

bdata = np.load(amass_sample, allow_pickle=True)
fps = 0
fps = bdata["mocap_framerate"]
frame_number = bdata["trans"].shape[0]

fId = 0
pose_seq = []
if bdata["gender"] == "male":
    bm = male_bm
else:
    bm = female_bm
    
down_sample = int(fps / ex_fps)

bodies = []

bdata_poses = bdata["poses"][::down_sample, ...]
bdata_trans = bdata["trans"][::down_sample, ...]
body_parms = {
    "root_orient": torch.Tensor(bdata_poses[:, :3]).cuda(),
    "pose_body": torch.Tensor(bdata_poses[:, 3:66]).cuda(),
    "pose_hand": torch.Tensor(bdata_poses[:, 66:]).cuda(),
    "trans": torch.Tensor(bdata_trans).cuda(),
    "betas": torch.Tensor(np.repeat(bdata["betas"][:num_betas][np.newaxis], repeats=len(bdata_trans), axis=0)).cuda(),
}

with torch.no_grad():
    body = bm(**body_parms)

vertices = body.v.detach().cpu().numpy()

In [12]:

trans_matrix = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]])

vertices2 = np.dot(vertices,trans_matrix)

def visualize_pose(vertices, ax, elev=20, azim=210, roll=0, vertical_axis="y"):
    ax.cla()  # Clear the previous plot
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=0.1, c="r")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.view_init(elev=elev, azim=azim, roll=roll, vertical_axis=vertical_axis)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

def update(frame):
    visualize_pose(vertices2[frame], ax)

ani = FuncAnimation(fig, update, frames=30, interval=50)  # 20 fps = 50ms interval
plt.close(fig)  # Prevents the static plot from showing

HTML(ani.to_jshtml())